In [9]:
# importing libraries
import numpy as np
import json
import zipfile
import pandas as pd
import re
import paralleldots
import seaborn as sns
import glob
from uclassify import uclassify

uclass = uclassify()

# copy/paste API key for ParallelDots
API_KEY = 'aSWrD1QSsUqdb8JYWJ6jMJMXrTxjFXKhdD4xQZKiUlQ'
READ_API_KEY = 'pUllRXrepgLH'

# setting API key for classifier
paralleldots.set_api_key(API_KEY)
uclass.setReadApiKey(READ_API_KEY)

In [4]:
# initializing lists
research_questions = {}
qtr = []
year = []
group = []
# special projects are marked with '_S' at the end of its name
special = []
word_counts = []
graph_counts = []
table_counts = []
index = 0

# reading zip file for a certain quarter
with zipfile.ZipFile("zips/FinalProjects-SP21-main.zip", "r") as f:
    for i, name in enumerate(f.namelist()):
        # skip files that are not jupyter notebooks
        if '.ipynb' not in name:
            continue
        rf = json.loads(f.read(name))
        word_count = 0
        graph_count = 0
        table_count = 0
        rq_found = False
        
        # loop through every cell
        for j, cell in enumerate(rf['cells']):
            if cell['cell_type'] == 'code':
                if ('outputs' in cell) and (cell['outputs'] != []):# and ('data' in cell['outputs'][0]):# and ('image/png' in cell['outputs'][0]['data'])
                    for k,_ in enumerate(cell['outputs']):
                        if 'data' in cell['outputs'][k]:
                            if 'image/png' in cell['outputs'][k]['data']:
                                graph_count += 1
                            if 'text/html' in cell['outputs'][k]['data']:
                                table_count += 1
            elif cell['cell_type'] == 'markdown':
                for c in cell['source']:
                    # if cell contains the research question header, update lists and extract the research question from the next cell
                    if (rq_found == False) and (re.search(r"(^#.*research question)", c.lower()) != None):
                        #print(\"RQ FOUND\")
                        
                        # extract quarter info from repo name
                        date = re.search(r"-(.*)-", name).group(1)
                        qtr.append(date[:2].upper())
                        year.append(int(date[2:]))

                        # extract group number
                        fname = re.search(r"/.*", name).group(0)
                        group.append(int(re.search(r"[$0-9^]{2,3}", fname)[0]))

                        special.append(False if re.search(r"_S\.ipynb", name) == None else True)
                        # get the research question from the cell AFTER the cell that contains the header 'Research Question'
                        rq = ' '.join(rf['cells'][j+1]['source'])
                        research_questions[str(group[index])+"_"+qtr[index]+"_"+str(year[index])] = rq
                        rq_found = True
                        index += 1
                    word_count += len(c.replace('#', '').lstrip().split(' '))
        if rq_found:
            word_counts.append(word_count)
            graph_counts.append(graph_count)
            table_counts.append(table_count)

In [13]:
# initialize topics list
topic = []

# loop through research questions and find the most confident topic class from topic classifier
for i, (g, q, y) in enumerate(zip(group, qtr, year)):
#     print(i, g, q, y, research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])])
#     response = paralleldots.taxonomy(research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])])
#     topic.append(response['taxonomy'][0]['tag'])
    d = uclass.classify(research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])],"Sentiment")
    print(d)
#     print("EXAMPLE:\n", research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])], " :: ", topic[i])

uClassifyError: "Authentification with the api key 'xxxxxxxxxgLH' for the classifier 'Sentiment' failed. The classifier doesn't exist."

In [11]:
uclass.classify?